In [58]:
import re

<link href="style.css" rel="stylesheet"></link>
<main>
<article class="day-desc"><h2>--- Day 17: Clumsy Crucible ---</h2><p>The lava starts flowing rapidly once the Lava Production Facility is operational. As you <span title="see you soon?">leave</span>, the reindeer offers you a parachute, allowing you to quickly reach Gear Island.</p>
<p>As you descend, your bird's-eye view of Gear Island reveals why you had trouble finding anyone on your way up: half of Gear Island is empty, but the half below you is a giant factory city!</p>
<p>You land near the gradually-filling pool of lava at the base of your new <em>lavafall</em>. Lavaducts will eventually carry the lava throughout the city, but to make use of it immediately, Elves are loading it into large <a href="https://en.wikipedia.org/wiki/Crucible" target="_blank">crucibles</a> on wheels.</p>
<p>The crucibles are top-heavy and pushed by hand. Unfortunately, the crucibles become very difficult to steer at high speeds, and so it can be hard to go in a straight line for very long.</p>
<p>To get Desert Island the machine parts it needs as soon as possible, you'll need to find the best way to get the crucible <em>from the lava pool to the machine parts factory</em>. To do this, you need to minimize <em>heat loss</em> while choosing a route that doesn't require the crucible to go in a <em>straight line</em> for too long.</p>
<p>Fortunately, the Elves here have a map (your puzzle input) that uses traffic patterns, ambient temperature, and hundreds of other parameters to calculate exactly how much heat loss can be expected for a crucible entering any particular city block.</p>
<p>For example:</p>
<pre><code>2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533
</code></pre>
<p>Each city block is marked by a single digit that represents the <em>amount of heat loss if the crucible enters that block</em>. The starting point, the lava pool, is the top-left city block; the destination, the machine parts factory, is the bottom-right city block. (Because you already start in the top-left block, you don't incur that block's heat loss unless you leave that block and then return to it.)</p>
<p>Because it is difficult to keep the top-heavy crucible going in a straight line for very long, it can move <em>at most three blocks</em> in a single direction before it must turn 90 degrees left or right. The crucible also can't reverse direction; after entering each city block, it may only turn left, continue straight, or turn right.</p>
<p>One way to <em>minimize heat loss</em> is this path:</p>
<pre><code>2<em>&gt;</em><em>&gt;</em>34<em>^</em><em>&gt;</em><em>&gt;</em><em>&gt;</em>1323
32<em>v</em><em>&gt;</em><em>&gt;</em><em>&gt;</em>35<em>v</em>5623
32552456<em>v</em><em>&gt;</em><em>&gt;</em>54
3446585845<em>v</em>52
4546657867<em>v</em><em>&gt;</em>6
14385987984<em>v</em>4
44578769877<em>v</em>6
36378779796<em>v</em><em>&gt;</em>
465496798688<em>v</em>
456467998645<em>v</em>
12246868655<em>&lt;</em><em>v</em>
25465488877<em>v</em>5
43226746555<em>v</em><em>&gt;</em>
</code></pre>
<p>This path never moves more than three consecutive blocks in the same direction and incurs a heat loss of only <code><em>102</em></code>.</p>
<p>Directing the crucible from the lava pool to the machine parts factory, but not moving more than three consecutive blocks in the same direction, <em>what is the least heat loss it can incur?</em></p>
</article>

</main>


In [65]:
from math import inf
from tabulate import tabulate
from heapq import heappop, heappush
from termcolor import colored


city = """
2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533
"""

test1 = """
2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
"""


def parse(city: str) -> list[list[int]]:
    return [[int(i) for i in l] for l in city.strip().splitlines()]


def turn_left(direction: tuple[int, int]) -> tuple[int, int]:
    if direction == (-1, 0):  # up
        return 0, -1  # left
    if direction == (1, 0):  # down
        return 0, 1  # right
    if direction == (0, 1):  # right
        return -1, 0  # up
    return 1, 0  # left -> down


def turn_right(direction: tuple[int, int]) -> tuple[int, int]:
    d_r, d_c = turn_left(direction)
    return -d_r, -d_c


def minimize_heat_loss(city: list[list[int]]) -> int:
    rows, cols = len(city), len(city[0])

    heap = []
    heappush(heap, (city[0][1], 0, (0, 1), (0, 1), [[(0, 1), (0, 1)]]))
    heappush(heap, (city[1][0], 0, (1, 0), (1, 0), [[(1, 0), (1, 0)]]))

    visited = set()

    while heap:
        heat_loss, straight_line, position, direction, path = heappop(heap)
        p_r, p_c = position
        if position == (rows - 1, cols - 1):
            return (
                heat_loss,
                path + [[position, direction]],
            )

        if (position, direction) in visited:
            continue

        visited.add((position, direction))

        d_r, d_c = direction
        if straight_line < 2 and 0 <= p_r + d_r < rows and 0 <= p_c + d_c < cols:
            heappush(
                heap,
                (
                    heat_loss + city[p_r + d_r][p_c + d_c],
                    straight_line + 1,
                    (p_r + d_r, p_c + d_c),
                    direction,
                    path + [[position, direction]],
                ),
            )

        d_r, d_c = turn_left(direction)
        if 0 <= p_r + d_r < rows and 0 <= p_c + d_c < cols:
            heappush(
                heap,
                (
                    heat_loss + city[p_r + d_r][p_c + d_c],
                    0,
                    (p_r + d_r, p_c + d_c),
                    (d_r, d_c),
                    path + [[position, direction]],
                ),
            )
        d_r, d_c = turn_right(direction)
        if 0 <= p_r + d_r < rows and 0 <= p_c + d_c < cols:
            heappush(
                heap,
                (
                    heat_loss + city[p_r + d_r][p_c + d_c],
                    0,
                    (p_r + d_r, p_c + d_c),
                    (d_r, d_c),
                    path + [[position, direction]],
                ),
            )
    return inf


# assert minimize_heat_loss(parse(city)) == 102


def print_path(city, path=True):
    city = parse(city)
    if path:
        hl, p = minimize_heat_loss(city)
        print(f"{hl=}")
    else:
        p = []

    for (p_r, p_c), d in p:
        city[p_r][p_c] = {(0, 1): ">", (0, -1): "<", (1, 0): "v", (-1, 0): "^"}[d]

    print()
    for line in city:
        for c in line:
            if isinstance(c, str):
                print(colored(c, "cyan"), end="")
            else:
                print(colored(c, "light_yellow"), end="")
        print()


print_path(city, False)
print_path(city)


2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533


TypeError: cannot unpack non-iterable float object

In [66]:
with open("./input/day17.txt") as f:
    print(print_path(f.read()))
# 1212 < anwer < 1253

TypeError: cannot unpack non-iterable float object